In [1]:
import time
from openpyxl import load_workbook
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException,StaleElementReferenceException, NoSuchElementException

In [2]:
def wdinit(obs='1'):
    Chrome_options = Options()
    not_windowed = False
    if obs == 'OFF' or obs == 0:
        Chrome_options.add_argument('--headless')
        Chrome_options.add_argument('--disable-gpu')
        not_windowed = True
    Chrome_options.add_argument("--disable-infobars")
    Chrome_options.add_argument("--incognito")
    Chrome_options.add_argument("--start-maximized")
    Chrome_options.add_experimental_option("prefs", {
        "safe browsing.enabled": True
    })

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Chrome_options)
    if not_windowed == True:
        print('WebDriver initiate successfully in Headless mode')
    else:
        print('WebDriver initiate successfully in Window mode')
    return driver

In [3]:
def login(driver, username, password, website = 'https://www.instagram.com/'):
    driver.get(website)

    try:
        username_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[name='username']"))
        )
        password_box = driver.find_element(By.CSS_SELECTOR, "input[name='password']")
        username_box.clear()
        password_box.clear()
        username_box.send_keys(username)
        password_box.send_keys(password)

        driver.find_element(By.CSS_SELECTOR, "button[type='submit']").submit()
        print("Login Successfully")

        time.sleep(2)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div._ac8f'))).click()
            print("Login Info Save Notification Found and click on Not Now")
        except (TimeoutException, NoSuchElementException):
            print("No Login Info Save Notification Found")

        time.sleep(2)

        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button._a9_1'))).click()
            print("Notification Found and click on Not Now")
        except (TimeoutException, NoSuchElementException):
            print("No Turn On Notification Found")

    except TimeoutException:
        print("Already logged in")

In [4]:
def searchnfill(driver, fill=""):
    try:
        driver.find_element(By.CSS_SELECTOR, 'svg[aria-label="Close').click()
    except NoSuchElementException:
        pass
    try:
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Search input']")))
    except (NoSuchElementException, TimeoutException):
        search_button = driver.find_element(By.CSS_SELECTOR, "svg[aria-label='Search']")
        search_button.click()
        search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[aria-label='Search input']")))
    finally:
        search_bar.clear()
        search_bar.send_keys(fill)
        time.sleep(1)

def searchresult(driver, searchKeyword):
    searchnfill(driver, searchKeyword)
    profilelist = list()
    for _ in range(3):
        try:
            profileelement = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.xocp1fn")))
            WebDriverWait(profileelement, 10).until(EC.presence_of_all_elements_located((By.XPATH, "./a")))
            profilelist = profileelement.find_elements(By.XPATH, "./a")
            break
        except (StaleElementReferenceException, NoSuchElementException, TimeoutException):
            time.sleep(1)
    profile_searched = len(profilelist)
    return profilelist, profile_searched

In [5]:
# Function to Search & Open Profile
def openprofile(driver, profilename):
    profilelist, total_profile = searchresult(driver, profilename)
    if len(profilelist)>0:
        if total_profile>0:
            for i in profilelist:
                spantag = i.find_elements(By.XPATH, './div/div/div/div[2]/div/div/span')
                l = len(spantag)
                if l == 1:
                    userid = i.find_element(By.XPATH, './div/div/div/div[2]/div/div/div/span').text
                    username = i.find_element(By.XPATH, './div/div/div/div[2]/div/div/span/span').text
                elif l == 2:
                    userid = i.find_element(By.XPATH, './div/div/div/div[2]/div/div/span[1]/span').text
                    username = "tag"
                if userid == profilename or username.startswith(profilename):
                    found_profile = i.find_element(By.XPATH, './div')
                    found_profile.click()
                    status = True
                    break
                else:
                    status = False
            driver.find_element(By.CSS_SELECTOR, "svg[aria-label='Search']").click()
        else:
            status = False
        time.sleep(1)
        return status, userid, username
    else:
        status = False
        username = None
        userid = None
        # print(userid,username)
        return status, userid, username

In [6]:
def post_info(driver, user_id, post_df, no_of_post = 2, total_post=0):
    post_element = driver.find_elements(By.CSS_SELECTOR, 'div._aabd')
    loop = min(total_post,no_of_post)
    while len(post_element)<=loop:
        driver.execute_script('window.scrollBy(0, 500);')
        post_element = driver.find_elements(By.CSS_SELECTOR, 'div._aabd')
    count = 1
    for i in post_element:
        post_url = caption = like_count = post_date = tagged_person = None
        if count == no_of_post+1:
            break
        for _ in range(3):
            try:
                post_url = i.find_element(By.XPATH,'./a').get_attribute('href')
                break
            except NoSuchElementException:
                pass
        i.click()
        time.sleep(1)
        for _ in range(3):
            try:
                caption_element = driver.find_element(By.CLASS_NAME, "_a9zn").find_element(By.CSS_SELECTOR, 'h1._ap3a')
                caption = caption_element.text
                break
            except NoSuchElementException:
                pass
        if caption is not None :
            captions_lines = caption.splitlines()
            captions = [line for line in captions_lines if line.strip() != '']
            caption = '\n'.join(captions)
        for _ in range(3):
            try:
                like_count_element = driver.find_element(By.CLASS_NAME, "_ae2s").find_element(By.XPATH,'./section[2]/div/div/span/a/span/span')
                like_count = int(like_count_element.text.replace(",",""))
                break
            except NoSuchElementException:
                pass
        for _ in range(3):
            try:
                post_date_element = driver.find_element(By.CLASS_NAME, "_ae2s").find_element(By.XPATH,'./div[2]/div/div/a/span/time')
                post_date = post_date_element.get_attribute('title')
                break
            except NoSuchElementException:
                pass
        if caption is not None :
            words = caption.split()
            tagged_list = [word for word in words if word.startswith('@')]
            if len(tagged_list) > 0:
                tagged_person = ', '.join(tagged_list)
        time.sleep(1)
        post_df.loc[len(post_df)] = [user_id, post_url, caption, like_count, post_date, tagged_person]
        count+=1
        try:
            post_close_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'svg[aria-label="Close"]')))
            post_close_element.click()
        except (NoSuchElementException,TimeoutException):
            pass
    # print(post_url)
    # print(caption)
    # print(like_count)
    # print(post_date)
    # print(tagged_person)
    return post_df

In [7]:
def info(driver, profilename, no_of_posts):
    profiles_df = pd.DataFrame(columns=['UserID', 'UserName', 'Bio', 'Public-Private', 'TotalPost', 'TotalFollowers', 'TotalFollowings', 'Verified'])
    post_df = pd.DataFrame(columns=['UserID', 'Post_URL', 'Caption', 'Like_Count','Post_Dated', 'All_Tagged'])
    for i in profilename:
        status, userid, username = openprofile(driver, i)
        confirm_userid = confirm_username = bio = pub_pri =  total_post = total_followers = total_followings = verified = None
        time.sleep(2)
        driver.refresh()
        if status == True:
            print(f"'{i}' is Found Successfully", end=" ")
            for _ in range(3):
                try:
                    userid_element = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2.x1lliihq')))
                    confirm_userid = userid_element.text
                    break
                except (NoSuchElementException, TimeoutException) :
                    pass
            for _ in range(3):
                try:
                    username_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div._aa_y')))
                    confirm_username = username_element.find_element(By.XPATH, './header/section/div[3]/div/span').text
                    break
                except (NoSuchElementException, TimeoutException) :
                    pass
            for _ in range(2):
                try:
                    verified_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'svg[aria-label="Verified"]')))
                    verified = 'Yes'
                    break
                except (NoSuchElementException, TimeoutException) :
                    verified = 'No'
            for _ in range(3):
                try:
                    private_account_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h2._aa_u')))
                    private_account =  private_account_element.text
                    if private_account == 'This Account is Private':
                        pub_pri = 'Private'
                        break
                except (NoSuchElementException, TimeoutException) :
                    pub_pri = 'Public'
                    try:
                        bio_element_1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1._ap3a')))
                        bio_1 = bio_element_1.text
                        try:
                            bio_element_2 = bio_element_1.find_element(By.XPATH, './br')
                            bio_2 = bio_element_2.text
                            bio = bio_1 + bio_2
                        except (NoSuchElementException, TimeoutException):
                            bio = bio_1
                    except (NoSuchElementException, TimeoutException):
                        bio = "No Bio"
            for _ in range(3):
                try:
                    profile_stat = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span._ac2a')))
                    total_post = int(profile_stat[0].find_element(By.XPATH, './span').text.replace(',', ''))
                    total_followers = int(profile_stat[1].get_attribute('title').replace(',', ''))
                    total_followings = int(profile_stat[2].find_element(By.XPATH, './span').text.replace(',', ''))
                    break
                except (NoSuchElementException, TimeoutException):
                    pass
            profiles_df.loc[len(profiles_df)] = [confirm_userid, confirm_username, bio, pub_pri, total_post, total_followers, total_followings, verified]
            print("and Profile statistics update successfully.")
            if pub_pri=='Public' and total_post > 0:
                post_info_dataframe = post_info(driver, confirm_userid, post_df, no_of_posts, total_post)
            else:
                if total_post == 0:
                    post_info_dataframe = 'No Post'
                else:
                    post_info_dataframe = "Profile is private, can't see the posts"
        else:
            print(f"'{i}' is not Found")
            profiles_df.loc[len(profiles_df)] = [i, confirm_username, bio, pub_pri, total_post, total_followers, total_followings, verified]
        # print("id", confirm_userid)
        # print("name", confirm_username)
        # print("verified",verified)
        # print("status", pub_pri)
        # print("bio", bio)
        # print("totalpost", total_post)
        # print("totalfollower",total_followers)
        # print("totalfollowing",total_followings)
    return profiles_df, post_info_dataframe

In [8]:
def logout(driver):
    if driver.current_url != 'https://www.instagram.com':
        try:
            setting_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'svg[aria-label="Settings"]')))
            setting_element.click()
            logout_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.xq9evs9'))).find_element(By.XPATH,'./div/div/div/div[6]/div')
            logout_element.click()
            print("Logout Successfully")
        except NoSuchElementException:
            pass
    else:
        print("Not Login or already Logout")

In [9]:
def wdclose(driver):
    driver.quit()
    print("Webdriver closed")

In [10]:
def save_to_excel(directory, profiles_info_dataframe, post_info_dataframe):
    os.chdir(directory)
    excel_file_name = 'database.xlsx'
    if not os.path.isfile(excel_file_name):
        with pd.ExcelWriter(excel_file_name, engine='openpyxl') as writer:
            profiles_info_dataframe.to_excel(writer, sheet_name='Profile_Information', index=False)
            post_info_dataframe.to_excel(writer, sheet_name='Post_Information', index=False)
    else:
        wb = load_workbook(excel_file_name)
        if 'Profile_Information' not in wb.sheetnames:
            wb.create_sheet(title='Profile_Information')
        if 'Post_Information' not in wb.sheetnames:
            wb.create_sheet(title='Post_Information')

        with pd.ExcelWriter(excel_file_name, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            profiles_info_dataframe.to_excel(writer, sheet_name='Profile_Information', index=False, header=True)
            post_info_dataframe.to_excel(writer, sheet_name='Post_Information', index=False, header=True)

In [11]:
user_name = 'sample'
pass_word = 'sample'

driver = wdinit(obs=0)
login(driver, user_name, pass_word)

WebDriver initiate successfully in Headless mode
Login Successfully
Login Info Save Notification Found and click on Not Now
No Turn On Notification Found


In [12]:
profile_list = ['aliaabhatt', 'So Delhi','china.travels','sachintendulkar','narendramodi']
profiles_information, post_information = info(driver, profile_list, 10)

'aliaabhatt' is Found Successfully and Profile statistics update successfully.
'So Delhi' is Found Successfully and Profile statistics update successfully.
'china.travels' is Found Successfully and Profile statistics update successfully.
'sachintendulkar' is Found Successfully and Profile statistics update successfully.
'narendramodi' is Found Successfully and Profile statistics update successfully.


In [13]:
profiles_information

,UserID,UserName,Bio,Public-Private,TotalPost,TotalFollowers,TotalFollowings,Verified
0,aliaabhatt,Alia Bhatt 💛,currently dreaming 💭,Public,2029,82854786,533,Yes
1,sodelhi,So Delhi,Bringing you the coolest things to do in Delhi...,Public,15510,640959,34,No
2,china.travels,China 🇨🇳 Asia Travel | Hotels | Food | Tips,🇨🇳 Discover the best of China\n💙 The leading C...,Public,9616,420639,1934,Yes
3,sachintendulkar,Sachin Tendulkar,Founder @sachintendulkarfoundation,Public,1248,47086459,49,Yes
4,narendramodi,Narendra Modi,Prime Minister of India 🇮🇳,Public,713,86782544,0,Yes


In [14]:
post_information

,UserID,Post_URL,Caption,Like_Count,Post_Dated,All_Tagged
0,aliaabhatt,https://www.instagram.com/p/C2_vvsRtHK8/,"beneath the silence, the forest reveals a dead...",97890,"Feb 6, 2024","@nimisha_sajayan, @roshan.matthew, @dibyenduof..."
1,aliaabhatt,https://www.instagram.com/p/C2ruktOPApq/,સરસ night at @filmfare 🫶🏻✨🖤\nP.S. this one’s f...,3409504,"Jan 29, 2024","@filmfare, @ranveersingh, @aapkadharam, @azmis..."
2,aliaabhatt,https://www.instagram.com/p/C2e0hcZvfWa/,❤️‍🔥\n#SanjayLeelaBhansali #RanbirKapoor @vick...,603128,"Jan 24, 2024","@vickykaushal09, @prerna_singh6, @bhansaliprod..."
3,aliaabhatt,https://www.instagram.com/p/C2XDWPbs3s9/,"To a night of culture, honour & cinema 🫶🏼✨\n@t...",2376570,"Jan 21, 2024","@turkialalshik, @rredfilms, @thebushramahdi"
4,aliaabhatt,https://www.instagram.com/p/C2MV3ozLvEh/,we are cuddly sleepers 💛💛💛💛💛\nre-united at las...,1468309,"Jan 17, 2024",@shaheenb
5,aliaabhatt,https://www.instagram.com/p/C1wiGcfvn4r/,Many moods & a million mirror selfies 💫,2482559,"Jan 6, 2024",None
6,aliaabhatt,https://www.instagram.com/p/C1jtmiOPJjP/,2024 to do - keep it wholesome & soul-some\nha...,3582159,"Jan 1, 2024",None
7,aliaabhatt,https://www.instagram.com/reel/C1enX3UsL1T/,2 0 2 3 🫶🏻,3192646,"Dec 30, 2023",None
8,aliaabhatt,https://www.instagram.com/p/C1Q91jcrv95/,grateful for this bunch.. grateful for so so m...,3697413,"Dec 25, 2023",None
9,aliaabhatt,https://www.instagram.com/p/C04JhH1PB1y/,Yellow there 👋🌼☀️,2370855,"Dec 15, 2023",None


In [15]:
logout(driver)

Logout Successfully


In [16]:
wdclose(driver)

Webdriver closed


In [17]:
location = r"location"
save_to_excel(location, profiles_information, post_information)